In [3]:
from dotenv import load_dotenv
from langchain import hub
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.embeddings import OllamaEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import END, StateGraph
from typing import Dict, TypedDict
from langchain.prompts import PromptTemplate
import pprint
import os
import chromadb
from sentence_transformers import SentenceTransformer
from typing import List

import os

# Set the environment variable
os.environ['USER_AGENT'] = 'chat/1.0 (pantomath42@nc.rr.com'

# Verify that it's set
print(os.environ.get('USER_AGENT'))

model = SentenceTransformer("all-MiniLM-L6-v2")

# Load environment variables
load_dotenv()

import logging

logging.basicConfig(level=logging.DEBUG)


run_local = 'Yes'
models = "openai"
openai_api_key = "Your_API_KEY"
google_api_key = "Your_API_KEY"
local_llm = 'llama3.1'
os.environ["TAVILY_API_KEY"] = ""
        

# Split documents
url  = 'https://lilianweng.github.io/posts/2023-06-23-agent/'
loader = WebBaseLoader(url)
docs = loader.load()


# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=100
)
all_splits = text_splitter.split_documents(docs)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/README.md HTTP/1.1" 200 0


chat/1.0 (pantomath42@nc.rr.com


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/sentence_bert_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/sentence-transformers/all-MiniLM-L6-v2/revision/main HTTP/1.1" 200 6130
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/sentence-transformers/all-MiniLM-L6-v2 HTTP/1.1" 200 6130
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): lilianweng.github.io:443
DEBUG:urllib3.connectionpool:https://l

In [5]:
all_splits [2]

Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performan

In [6]:
lst_ids = [f"chunk_{idx+1}" for idx, _ in enumerate(all_splits)]

In [8]:
len(all_splits)

33

In [10]:
lst_docs = [metadata for metadata in enumerate(all_splits)]

In [21]:
texts = [chunk.page_content for chunk in all_splits]

# Encode the text
embeddings = [model.encode(text).tolist() for text in texts]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [80]:
import os
#os.environ['TOKENIZERS_PARALLELISM'] = "false"
#os.environ['OPENAI_API_KEY'] = "noneed"
#os.environ['OPENAI_MODEL'] = "nomic-embed-text"
#os.environ['OPENAI_BASE_URL'] = "http://192.168.2.39:11434"

#def embed_function(texts):
#    return model.encode(texts).tolist()

#from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text", base_url="http://192.168.2.39:11434")

In [81]:
from langchain_community.vectorstores import Chroma
collection_name = "rag-chroma"

vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name=collection_name,
#    embedding=OpenAIEmbeddings()
#    embedding=OllamaEmbeddings()
    embedding= embeddings
)


INFO:chromadb.api.segment:Collection rag-chroma is not created.
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.2.39:11434
DEBUG:urllib3.connectionpool:Resetting dropped connection: us-api.i.posthog.com
DEBUG:urllib3.connectionpool:https://us-api.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 15
DEBUG:urllib3.connectionpool:http://192.168.2.39:11434 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.2.39:11434
DEBUG:urllib3.connectionpool:http://192.168.2.39:11434 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.2.39:11434
DEBUG:urllib3.connectionpool:http://192.168.2.39:11434 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.2.39:11434
DEBUG:urllib3.connectionpool:http://192.168.2.39:11434 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP conn

InvalidDimensionException: Embedding dimension 768 does not match collection dimensionality 4096

In [82]:
retriever = vectorstore.as_retriever()
print(retriever)


tags=['Chroma', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x23806ff80>
